In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

from context import rf_pool

In [ ]:
# get MNIST training data
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.MNIST(root='../data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='../data', train=False, download=True, transform=transform)

In [ ]:
# create trainloader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                          shuffle=True, num_workers=2)

In [ ]:
# initialize model
dbn = rf_pool.models.DeepBeliefNetwork()

In [ ]:
# create receptive field pooling layer for probabilistic max pooling (Lee et al., 2009)
rf_layer = rf_pool.pool.ProbmaxPool(2)

In [ ]:
# Set Multinomial log partition function
log_part_fn = lambda x : torch.log(1. + torch.nn.functional.lp_pool2d(torch.exp(x), 1, (2,2)))

In [ ]:
# append two RBM layers with sigmoid vis/hid activations
dbn.append(0, rf_pool.modules.RBM(hidden=torch.nn.Conv2d(1, 24, 11), activation=None, 
                                  pool=rf_layer, log_part_fn=log_part_fn,
                                  sample=torch.distributions.Bernoulli,
                                  vis_activation=torch.nn.Sigmoid(),
                                  vis_sample=torch.distributions.Bernoulli))
dbn.append(1, rf_pool.modules.RBM(hidden=torch.nn.Conv2d(24, 40, 9), activation=torch.nn.Sigmoid(), 
                                  pool=None, sample=torch.distributions.Bernoulli,
                                  vis_activation=torch.nn.Sigmoid(), 
                                  vis_sample=torch.distributions.Bernoulli))

In [ ]:
# train first layer with sparsity, monitor weights and negative examples
optim = torch.optim.SGD(dbn.layers['0'].parameters(), lr=1e-2)
loss_history0 = dbn.train_layer('0', 1, trainloader, optim, monitor=100, k=1,
                                persistent=torch.zeros(20,1,28,28),
                                show_weights={'layer_id': '0', 'cmap': 'gray'}, 
                                show_negative={'layer_id': '0', 'n_images': 5, 'cmap': 'gray'},
                                monitor_fn={'fn': dbn.layers['0'].pseudo_likelihood})

In [ ]:
# train second layer, monitor weights and negative examples
# note: second layer weights are projected to the image space
optim = torch.optim.SGD(dbn.layers['1'].parameters(), lr=1e-3)
loss_history1 = dbn.train_layer('1', 1, trainloader, optim, monitor=100, k=1,
                                persistent=torch.zeros(20,24,9,9),
                                show_weights={'layer_id': '1', 'cmap': 'gray'},
                                show_negative={'layer_id': '1', 'n_images': 5, 'cmap': 'gray'},
                                monitor_fn={'fn': dbn.layers['1'].pseudo_likelihood})